In [1]:
import numpy as np
import pandas as pd

data= pd.read_csv("dat.csv", sep=",")
# data2= pd.read_csv("dataset2.csv", sep=",")


In [2]:
data = data.dropna()
data.shape

(2195, 117)

In [3]:
y = np.array(data.avgprice).astype(int)
y

array([ 35, 147, 197, ..., 584, 212, 199])

In [4]:
data.columns

Index(['Unnamed: 0', 'id', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost',
       ...
       'avgprice', 'pricevar', 'counteng', 'pos', 'neg', 'zero', 'wordcount',
       'charcount', 'sentiment', 'count'],
      dtype='object', length=117)

In [5]:

datafilter = data.drop(['Unnamed: 0','amenities','amenities_list',"amenities_list", 
                                   "requires_license", "host_thumbnail_url",
                                   "host_picture_url", "picture_url"],1)
#                         "id","medium_url","picture_url","xl_picture_url","host_thumbnail_url"],1)

In [6]:

import math

def maxinfo(data):
    entropy = np.zeros(data.shape[1])
    for i in range(data.shape[1]):
#         print(i)
        entropy[i] = _entropy(data, i)
        
    return entropy

def _entropy(data, target_attr):
 
    val_freq = data.iloc[:,target_attr].value_counts()
    data_entropy = 0.0

    # Calculate the entropy of the data for the target attribute
    for freq in val_freq:
        data_entropy += (-freq/len(data)) * math.log(freq/len(data), 2) 

    return data_entropy

def gain(data, attr, target_attr):
 
    subset_entropy = 0.0
 
    val_freq = data.iloc[:,target_attr].value_counts()
 
    # Calculate the sum of the entropy for each subset of records weighted by their probability of occuring in the training set.
    for val in val_freq.keys():
        val_prob = freq/len(data)
        data_subset = [record for record in data if record[attr] == val]
        subset_entropy += val_prob * entropy(data_subset, target_attr)
 
    # Subtract the entropy of the chosen attribute from the entropy of the whole data set with respect to the target attribute (and return it)
    return (entropy(data, target_attr) - subset_entropy)

In [7]:
# entropy=maxinfo(datafilter)
# ind =np.argsort(entropy)[0:100]
# bestdata = datafilter.iloc[:,ind]
bestdata = datafilter
filterlast = bestdata.select_dtypes(exclude=[object])
totransform = bestdata.select_dtypes(include=[object])
if len(totransform) == 0:
    final = filterlast
else:
    dummy = pd.get_dummies(totransform)
    final = pd.concat([filterlast,dummy],axis=1)
dataSize = len(y)
ind = np.random.permutation(dataSize)


"80-20 split"
nTest = int(0.20 * dataSize)
test_indices = ind[:nTest]
train_indices = ind[nTest:]

train_set_x = final.iloc[train_indices,:]
train_set_y = y[train_indices]
test_set_x = final.iloc[test_indices,:]
test_set_y = y[test_indices]

In [112]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
range1 = np.zeros(50)
range2 = np.zeros(50)

for i in range(50):
    forest = RandomForestClassifier( n_estimators = 100, max_depth=None, min_samples_split=10)
#                                     max_depth=None, min_samples_split=10, 
#                                     min_samples_leaf=5, min_weight_fraction_leaf=0.0, max_features='auto',
#                                     max_leaf_nodes=None, min_impurity_split=1e-07, bootstrap=True, oob_score=True,
#                                     n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight=None)

    # forest = RandomForestClassifier( n_estimators = 1000)

    print ("Fitting a random forest to labeled training data...")
    forest = forest.fit( train_set_x, train_set_y )
    print(np.sqrt(np.mean(forest.predict(train_set_x)-train_set_y)**2))
    range2[i] = np.sqrt(np.mean(forest.predict(train_set_x)-train_set_y)**2)

    # print (np.mean(cross_val_score(forest, train_set_x.as_matrix(), train_set_y.as_matrix(), cv=10)))
    # Test & extract results 
    result = forest.predict( test_set_x )
    print(np.sqrt(np.mean(result-test_set_y)**2))
    range1[i] = np.sqrt(np.mean(result-test_set_y)**2)
    
#     results.append(np.mean(result==test_set_y))

Fitting a random forest to labeled training data...
0.144646924829
8.74031890661
Fitting a random forest to labeled training data...
0.0438496583144
10.4054669704
Fitting a random forest to labeled training data...
0.211275626424
13.8063781321
Fitting a random forest to labeled training data...
0.345671981777
13.7266514806
Fitting a random forest to labeled training data...
0.150911161731
13.7061503417
Fitting a random forest to labeled training data...
0.173120728929
9.81548974943
Fitting a random forest to labeled training data...
0.248861047836
12.840546697
Fitting a random forest to labeled training data...
0.0808656036446
12.7539863326
Fitting a random forest to labeled training data...
0.28416856492
10.8291571754
Fitting a random forest to labeled training data...
0.0620728929385
9.96127562642
Fitting a random forest to labeled training data...
0.303530751708
14.1161731207
Fitting a random forest to labeled training data...
0.125284738041
12.2232346241
Fitting a random forest to 

In [113]:
from sklearn.ensemble import RandomForestClassifier
range3 = np.zeros(50)
range4 = np.zeros(50)

for i in range(50):
    forest = RandomForestClassifier( n_estimators = 100)
    forest = forest.fit( train_set_x, train_set_y )
    print ("Fitting a random forest to labeled training data...")
    forest = forest.fit( train_set_x, train_set_y )
    print(np.sqrt(np.mean(forest.predict(train_set_x)-train_set_y)**2))
    range4[i] = np.sqrt(np.mean(forest.predict(train_set_x)-train_set_y)**2)

    # print (np.mean(cross_val_score(forest, train_set_x.as_matrix(), train_set_y.as_matrix(), cv=10)))
    # Test & extract results 
    result = forest.predict( test_set_x )
    print(np.sqrt(np.mean(result-test_set_y)**2))
    #     results.append(np.mean(result==test_set_y))
    range3[i] = np.sqrt(np.mean(result-test_set_y)**2)



Fitting a random forest to labeled training data...
0.0
15.4054669704
Fitting a random forest to labeled training data...
0.0
13.1936218679
Fitting a random forest to labeled training data...
0.0
14.0865603645
Fitting a random forest to labeled training data...
0.0
11.8883826879
Fitting a random forest to labeled training data...
0.0
12.3917995444
Fitting a random forest to labeled training data...
0.0
8.49430523918
Fitting a random forest to labeled training data...
0.0
15.9293849658
Fitting a random forest to labeled training data...
0.0
16.006833713
Fitting a random forest to labeled training data...
0.0
13.2186788155
Fitting a random forest to labeled training data...
0.0
14.0455580866
Fitting a random forest to labeled training data...
0.0
13.277904328
Fitting a random forest to labeled training data...
0.0
13.979498861
Fitting a random forest to labeled training data...
0.0
15.3849658314
Fitting a random forest to labeled training data...
0.0
11.5671981777
Fitting a random forest

In [114]:
print(np.mean(range1),np.mean(range2))
print(np.std(range1),np.std(range2))
print(np.mean(range3),np.mean(range4))


12.0458769932 0.188029612756
1.60677900941 0.110079045517
14.138952164 0.0


In [97]:
 data[["requires_license", "host_thumbnail_url", "host_picture_url", "picture_url"]].mean()

requires_license      0.0
host_thumbnail_url    1.0
host_picture_url      1.0
picture_url           1.0
dtype: float64

In [101]:
ind =np.argsort(entropy)[0:50]
bestdata =datafilter.iloc[:,ind]

In [89]:
entropy=maxinfo(datafilter)


In [91]:
np.argsort(entropy)[0:50]

array([ 44,  10,   3,   9,  95, 103,  14, 102,  94,  96,  62,  47, 100,
        67,  98,  97,  92, 101,  25,  90,  55,  66,  63,  88,  48,  87,
        80,  89,  18,  65,  78,  86,  58,   8,  71,  54,  45,  99,  53,
        77,  57,  73,   1,   2,   4,  15,  93,  76,  51,  52], dtype=int64)

In [74]:
val_freq =datafilter.iloc[:,0].value_counts()

In [17]:
for i in range(50):
    forest = RandomForestClassifier( n_estimators = 100)
    forest = forest.fit( train_set_x, train_set_y )
    print ("Fitting a random forest to labeled training data...")
    forest = forest.fit( train_set_x, train_set_y )
    print(np.sqrt(np.mean(forest.predict(train_set_x)-train_set_y)**2))
#     range4[i] = np.sqrt(np.mean(forest.predict(train_set_x)-train_set_y)**2)

    # print (np.mean(cross_val_score(forest, train_set_x.as_matrix(), train_set_y.as_matrix(), cv=10)))
    # Test & extract results 
    result = forest.predict( test_set_x )
    print(np.sqrt(np.mean(result-test_set_y)**2))
    #     results.append(np.mean(result==test_set_y))
#     range3[i] = np.sqrt(np.mean(result-test_set_y)**2)

Fitting a random forest to labeled training data...
0.0
16.3439635535
Fitting a random forest to labeled training data...
0.0
16.0820045558
Fitting a random forest to labeled training data...
0.0
13.2551252847
Fitting a random forest to labeled training data...
0.0
16.6719817768
Fitting a random forest to labeled training data...
0.0
14.7403189066
Fitting a random forest to labeled training data...
0.0
15.1822323462
Fitting a random forest to labeled training data...
0.0
14.6104783599
Fitting a random forest to labeled training data...
0.0
14.4806378132
Fitting a random forest to labeled training data...
0.0
16.7790432802
Fitting a random forest to labeled training data...
0.0
12.9977220957
Fitting a random forest to labeled training data...
0.0
16.1845102506
Fitting a random forest to labeled training data...
0.0
13.0182232346
Fitting a random forest to labeled training data...
0.0
13.3621867882
Fitting a random forest to labeled training data...
0.0
17.3986332574
Fitting a random for

In [15]:
bestdata.iloc[2,:]

id                                      6695
thumbnail_url                              1
medium_url                                 1
xl_picture_url                             1
host_response_time              within a day
host_response_rate                       100
host_acceptance_rate                      90
host_is_superhost                          0
host_listings_count                        7
host_total_listings_count                  7
host_verifications                         4
host_has_profile_pic                       1
host_identity_verified                     1
neighbourhood_cleansed               Roxbury
zipcode                                02119
is_location_exact                          1
property_type                      Apartment
room_type                    Entire home/apt
accommodates                               4
bathrooms                                  1
bedrooms                                   1
beds                                       2
bed_type  

In [114]:
final

,id,thumbnail_url,medium_url,picture_url,xl_picture_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,...,zero,wordcount,charcount,sentiment,count,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed
0,3353,1,1,1,1,within a few hours,90.0,78.0,0,1,...,0.000000,68.250000,369.750000,0.281329,34,0,0,0,0,1
1,3353,1,1,1,1,within a few hours,90.0,78.0,0,1,...,0.000000,68.250000,369.750000,0.281329,34,0,0,0,0,1
2,3353,1,1,1,1,within a few hours,90.0,78.0,0,1,...,0.000000,68.250000,369.750000,0.281329,34,0,0,0,0,1
3,3353,1,1,1,1,within a few hours,90.0,78.0,0,1,...,0.000000,68.250000,369.750000,0.281329,34,0,0,0,0,1
4,3353,1,1,1,1,within a few hours,90.0,78.0,0,1,...,0.000000,68.250000,369.750000,0.281329,34,0,0,0,0,1
5,3353,1,1,1,1,within a few hours,90.0,78.0,0,1,...,0.000000,68.250000,369.750000,0.281329,34,0,0,0,0,1
6,3353,1,1,1,1,within a few hours,90.0,78.0,0,1,...,0.000000,68.250000,369.750000,0.281329,34,0,0,0,0,1
7,3353,1,1,1,1,within a few hours,90.0,78.0,0,1,...,0.000000,68.250000,369.750000,0.281329,34,0,0,0,0,1
8,3353,1,1,1,1,within a few hours,90.0,78.0,0,1,...,0.000000,68.250000,369.750000,0.281329,34,0,0,0,0,1
9,3353,1,1,1,1,within a few hours,90.0,78.0,0,1,...,0.000000,68.250000,369.750000,0.281329,34,0,0,0,0,1
